## Pipeline

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import imutils

class CropBrainTumorLayer(Layer):
    def __init__(self, **kwargs):
        super(CropBrainTumorLayer, self).__init__(**kwargs)

    def call(self, inputs):
        def crop_brain_tumor(image):
            image = tf.cast(image, tf.uint8)
            image = image.numpy()
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (5, 5), 0)
            thres = cv2.threshold(gray, 45, 224, cv2.THRESH_BINARY)[1]
            thres = cv2.erode(thres, None, iterations=2)
            thres = cv2.dilate(thres, None, iterations=2)
            cnts = cv2.findContours(thres.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            c = max(cnts, key=cv2.contourArea)
            extLeft = tuple(c[c[:, :, 0].argmin()][0])
            extRight = tuple(c[c[:, :, 0].argmax()][0])
            extTop = tuple(c[c[:, :, 1].argmin()][0])
            extBot = tuple(c[c[:, :, 1].argmax()][0])
            new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]
            new_image = cv2.resize(new_image, (224, 224))
            return new_image.astype(np.float32) / 255.0

        def py_crop_brain_tumor(image):
            return tf.py_function(crop_brain_tumor, [image], tf.float32)

        return tf.map_fn(py_crop_brain_tumor, inputs, dtype=tf.float32)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 224, 224, 3)

# Instantiate the custom layer
crop_layer = CropBrainTumorLayer()
# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
# Define the model pipeline
model_pipeline = Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    crop_layer,
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  
])

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model_pipeline.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model_pipeline.summary()

/Users/tsheltrimpemo/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ crop_brain_tumor_layer          │ (None, 224, 224, 3)    │             0 │
│ (CropBrainTumorLayer)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,846,273 (56.63 MB)

 Trainable params: 131,585 (514.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

## Save Model

In [33]:
model_pipeline.save('model.h5')